# LLM Fine-Tuning for API Calling (Course Mini Project)

This project compares a base small language model with a fine-tuned version
on API calling tasks using the **APIBench** dataset
(https://huggingface.co/datasets/gorilla-llm/APIBench).

The goal is to evaluate whether supervised fine-tuning improves
structured API call generation.

## Hypothesis

Fine-tuning a small language model on APIBench API calling data
will generate more accurate and syntactically valid API calls
than the base (non-fine-tuned) model.

**Baseline:** Qwen-2.5-1.5B without task-specific fine-tuning   (Resuts in baseline_qwen.py file in this repository)

**Fine-tuned:** Qwen-2.5-1.5B fine-tuned using LoRA (PEFT)


In [ ]:
!pip install bitsandbytes
!pip install evaluate
!pip install sacrebleu rouge-score


In [ ]:
import os
import json
import random
import math

import torch
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, PeftModel
import bitsandbytes as bnb

print("bitsandbytes:", bnb.__version__)


bitsandbytes: 0.48.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
DATA_PATH = "/content/drive/MyDrive/API-Pack-Dataset/api_pack_starcoder_training.jsonl"

dataset = []
with open(DATA_PATH, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            dataset.append(json.loads(line))

print("Total raw samples:", len(dataset))


Total raw samples: 1014093


In [ ]:
def build_prompt(ex):
    inp = ex["input"]

    lang = inp.get("lang", None)
    if lang is None:
        raise ValueError("Record without valid 'lang' should have been skipped.")

    return (
        f"You are an API client code generator. "
        f"You MUST output ONLY valid {lang} code. "
        f"No comments. No explanations. No markdown. Only raw code.\n\n"
        f"### USER REQUEST:\n{inp.get('instruction','')}\n\n"
        f"### ENDPOINT PATH:\n{inp.get('path','')}\n\n"
        f"### DESCRIPTION:\n{inp.get('description','')}\n\n"
        f"### PARAMETERS:\n{inp.get('api_arguments',{})}\n\n"
        f"### OUTPUT:\n"
    )


In [ ]:
python_rows = []
other_rows = []

for obj in dataset:
    inp = obj.get("input", {})
    out = obj.get("output", {})


    raw_lang = inp.get("lang", None)
    if raw_lang is None:
        continue

    lang = str(raw_lang).strip()

    prompt = build_prompt(obj)
    api_call = out.get("api_call", "")

    row = {"text": prompt + api_call}

    if lang.lower() == "python":
        python_rows.append(row)
    else:
        other_rows.append(row)

print("Python samples:", len(python_rows))
print("Non-Python samples:", len(other_rows))


Python samples: 100860
Non-Python samples: 913233


In [ ]:
random.shuffle(python_rows)
test_rows = python_rows[:100]
python_rows = python_rows[100:]

print("Test samples (Python only):", len(test_rows))
print("Remaining Python for training:", len(python_rows))


Test samples (Python only): 100
Remaining Python for training: 100760


In [ ]:
LIMIT = 200_000

train_rows = []
train_rows.extend(python_rows)

if len(train_rows) < LIMIT:
    needed = LIMIT - len(train_rows)
    random.shuffle(other_rows)
    train_rows.extend(other_rows[:needed])

train_rows = train_rows[:LIMIT]

print("Final train samples:", len(train_rows))

Final train samples: 200000


In [ ]:
train_ds = Dataset.from_list(train_rows)
test_ds  = Dataset.from_list(test_rows)

ds = DatasetDict({
    "train": train_ds,
    "test": test_ds
})


In [ ]:

print(ds)


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 200000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 100
    })
})


In [ ]:
model_name = "Qwen/Qwen2.5-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=1024,
    )

tokenized_ds = ds.map(tokenize_fn, batched=True, remove_columns=["text"])
tokenized_ds.set_format("torch")

train_tokenized = tokenized_ds["train"]
test_tokenized  = tokenized_ds["test"]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/200000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:

dtype = (
    torch.bfloat16
    if torch.cuda.is_available() and torch.cuda.is_bf16_supported()
    else torch.float16
)

base = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=dtype,
    trust_remote_code=True,
)
base.config.use_cache = False

lora_cfg = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(base, lora_cfg)
model.print_trainable_parameters()

collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

trainable params: 4,358,144 || all params: 1,548,072,448 || trainable%: 0.2815


In [ ]:
OUTPUT_DIR = "/content/drive/MyDrive/Qwen-FINAL-RUN"
os.makedirs(OUTPUT_DIR, exist_ok=True)

first_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    max_steps=100,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    eval_strategy = "epoch",

    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,

    logging_steps=50,
    bf16=True,
    optim="paged_adamw_32bit",
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=first_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=collator,
)

print("=== TRAINING 100 STEPS ===")
trainer.train()


/tmp/ipython-input-2943207292.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


=== TRAINING 100 STEPS ===


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sinhaano (sinhaano-usc) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
0,0.975600,0.905399


TrainOutput(global_step=100, training_loss=1.108980827331543, metrics={'train_runtime': 191.7297, 'train_samples_per_second': 2.086, 'train_steps_per_second': 0.522, 'total_flos': 3231003652915200.0, 'train_loss': 1.108980827331543, 'epoch': 0.002})

In [ ]:
RESUME_CKPT = f"{OUTPUT_DIR}/checkpoint-550"

base2 = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=dtype,
    trust_remote_code=True,
)

model2 = PeftModel.from_pretrained(
    base2,
    RESUME_CKPT,
    is_trainable=True,
)

full_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,


    eval_strategy="epoch",
    eval_steps = 200,


    save_strategy="steps",
    save_steps=2000,
    save_total_limit=5,

    logging_steps=50,
    bf16=True,
    optim="paged_adamw_32bit",
    remove_unused_columns=False,
)

trainer2 = Trainer(
    model=model2,
    args=full_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    data_collator=collator,
)

/tmp/ipython-input-767544966.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer2 = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


In [ ]:
print("=== RESUMING FOR FULL EPOCH TRAINING ===")
trainer2.train(resume_from_checkpoint=RESUME_CKPT)

print("=== TRAINING COMPLETE ===")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


=== RESUMING FOR FULL EPOCH TRAINING ===


	eval_steps: 200 (from args) != 500 (from trainer_state.json)
	save_steps: 2000 (from args) != 50 (from trainer_state.json)


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
for i in range(5):
    sample = train_tokenized[i]["input_ids"]
    non_pad = sum([1 for x in sample if x != tokenizer.pad_token_id])
    print(i, "tokens =", non_pad)
    print(tokenizer.decode(sample))
    print("-----")


0 tokens = 205
You are an API client code generator. You MUST output ONLY valid Python code. No comments. No explanations. No markdown. Only raw code.

### USER REQUEST:
I'd like to use the Data-Who Covid 19 Data-API to ensure I have an up-to-date list of valid country and territory names for my project. How can I retrieve this information using the API's "names" functionality?

### ENDPOINT PATH:
/api/data/names

### DESCRIPTION:
Get a list of valid country and territory names.

### PARAMETERS:
{}

### OUTPUT:
import http.client

conn = http.client.HTTPConnection("undefinedhttps")

headers = {
    'X-RapidAPI-Key': "SOME_STRING_VALUE",
    'X-RapidAPI-Host': "SOME_STRING_VALUE"
    }

conn.request("GET", "//who-covid-19-data.p.rapidapi.com/api/data/names", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>

In [ ]:
import json
with open(f"{RESUME_CKPT}/trainer_state.json") as f:
    state = json.load(f)
state["global_step"], state["max_steps"]


(350, 50000)

In [ ]:
print(trainer2.state.log_history[-5:])


[{'loss': 0.5646, 'grad_norm': 0.5046700835227966, 'learning_rate': 0.00012980400000000002, 'epoch': 0.351, 'step': 17550}, {'loss': 0.5912, 'grad_norm': 0.5521150231361389, 'learning_rate': 0.00012960400000000001, 'epoch': 0.352, 'step': 17600}, {'loss': 0.6512, 'grad_norm': 0.6232796311378479, 'learning_rate': 0.000129404, 'epoch': 0.353, 'step': 17650}, {'loss': 0.5864, 'grad_norm': 0.7386950254440308, 'learning_rate': 0.000129204, 'epoch': 0.354, 'step': 17700}, {'loss': 0.6374, 'grad_norm': 0.5235446691513062, 'learning_rate': 0.00012900400000000003, 'epoch': 0.355, 'step': 17750}]


#EVAL

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

BASE_MODEL = "Qwen/Qwen2.5-1.5B"
LORA_DIR = "/content/drive/MyDrive/Qwen-FINAL-RUN/checkpoint-17750"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)

base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model = PeftModel.from_pretrained(
    base,
    LORA_DIR,
)

model.eval()


In [ ]:
def generate_code(prompt, max_new_tokens=512):

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1536,
    ).to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.3,
        top_p=0.95,
        repetition_penalty=1.0,
        pad_token_id=tokenizer.eos_token_id,
    )

    generated_ids = output[0][inputs.input_ids.shape[1]:]
    generated = tokenizer.decode(generated_ids, skip_special_tokens=True)

    return generated.strip()

In [ ]:
import evaluate
import math
import torch

bleu_metric = evaluate.load("bleu")
sacrebleu_metric = evaluate.load("sacrebleu")
rouge_metric = evaluate.load("rouge")


In [ ]:
def exact_match(pred, gold):
    return int(pred.strip() == gold.strip())


In [ ]:
def compute_perplexity(text):
    enc = tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        loss = model(**enc, labels=enc["input_ids"]).loss
    return math.exp(loss.item())


In [ ]:
import ast

def python_is_valid(code):
    try:
        ast.parse(code)
        return True
    except:
        return False


In [ ]:
import random

python_rows = []
other_rows = []

for obj in dataset:
    inp = obj.get("input", {})
    out = obj.get("output", {})

    raw_lang = inp.get("lang", None)
    if raw_lang is None:
        continue

    lang = str(raw_lang).strip().lower()

    prompt = build_prompt(obj)
    api_call = out.get("api_call", "")

    row = {"prompt": prompt, "api_call": api_call, "full_obj": obj}

    if lang == "python":
        python_rows.append(row)
    else:
        other_rows.append(row)

print(f"Total Python samples found: {len(python_rows)}")

random.shuffle(python_rows)
held_out_test_rows = python_rows[:100]
python_rows = python_rows[100:]

print(f"Held-out REAL test set (never seen): {len(held_out_test_rows)}")
print(f"Python samples that went into training : {len(python_rows)}")

eval_samples = held_out_test_rows



Total Python samples found: 100860
Held-out REAL test set (never seen): 100
Python samples that went into training : 100760


## Evaluation

Models are evaluated on a held-out set of Python APIBench samples
that were not used during training.

Metrics:
- Exact Match
- BLEU, SacreBLEU, ROUGE-L
- Syntax validity (Python parseable)
- Perplexity


## Decoding Notes

This notebook includes two decoding modes:

- **Deterministic (do_sample=False):** used for the reported metrics so results are reproducible.
- **Sampling (do_sample=True):** used for qualitative examples to observe output diversity.


In [ ]:
print("\n=== RUNNING EVALUATION ON 100 REAL HELD-OUT SAMPLES ===\n")

results = {
    "exact_match": [], "bleu": [], "sacrebleu": [], "rougeL": [], "perplexity": [], "syntax": []
}

for ex in eval_samples:
    original_obj = ex["full_obj"]
    gold = ex["api_call"]
    prompt = build_prompt(original_obj)
    pred = generate_code(prompt)

    results["exact_match"].append(exact_match(pred, gold))

    results["bleu"].append(bleu_metric.compute(predictions=[pred], references=[[gold]])["bleu"])

    results["sacrebleu"].append(sacrebleu_metric.compute(predictions=[pred], references=[gold])["score"])


    results["rougeL"].append(rouge_metric.compute(predictions=[pred], references=[gold])["rougeL"])


    results["perplexity"].append(compute_perplexity(pred))

    results["syntax"].append(int(python_is_valid(pred)))

print("=== FINAL HONEST METRICS (100 held-out Python samples) ===")
print(f"Exact Match     : {mean(results['exact_match']):.4f}")
print(f"BLEU            : {mean(results['bleu']):.4f}")
print(f"SacreBLEU       : {mean(results['sacrebleu']):.4f}")
print(f"ROUGE-L         : {mean(results['rougeL']):.4f}")
print(f"Perplexity      : {mean(results['perplexity']):.4f}")
print(f"Syntax Valid    : {mean(results['syntax']):.4f}")


=== RUNNING EVALUATION ON 100 REAL HELD-OUT SAMPLES ===

=== FINAL HONEST METRICS (100 held-out Python samples) ===
Exact Match     : 0.2100
BLEU            : 0.7357
SacreBLEU       : 73.5684
ROUGE-L         : 0.7987
Perplexity      : 2.1119
Syntax Valid    : 0.8800


In [ ]:
!ls -R /content/drive/MyDrive/Qwen-FINAL-RUN/


In [ ]:
print("\n" + "="*80)
print("SHOWING 5 DIVERSE EXAMPLES: PREDICTION vs GROUND TRUTH")
print("="*80 + "\n")


for idx, ex in enumerate(eval_samples[:5]):
    original_obj = ex["full_obj"]
    gold = ex["api_call"].strip()

    prompt = build_prompt(original_obj)
    pred = generate_code(prompt).strip()

    em = pred == gold
    syntax_ok = python_is_valid(pred)

    print(f"SAMPLE {idx+1} | Exact Match: {em} | Syntax Valid: {syntax_ok}")
    print("-" * 70)
    print("USER INSTRUCTION:")
    print(original_obj["input"].get("instruction", "").strip())
    print("\nENDPOINT:", original_obj["input"].get("path", "").strip())
    print("\nPREDICTED CODE:")
    print(pred)
    print("\nGROUND TRUTH CODE:")
    print(gold)
    print("\n" + "="*80 + "\n")


SHOWING 5 DIVERSE EXAMPLES: PREDICTION vs GROUND TRUTH

SAMPLE 1 | Exact Match: False | Syntax Valid: True
----------------------------------------------------------------------
USER INSTRUCTION:
Please give me an example of how to use the endpoint getTokenHolders from Neblio REST API Suite to find out which addresses hold a specific token and how many tokens they have.

ENDPOINT: /ntp1/stakeholders/{tokenid}

PREDICTED CODE:
import http.client

conn = http.client.HTTPConnection("ntp1node.nebulao.io")

headers = { 'X-Nebulaplugin-Id': "REPLACE_KEY_VALUE" }

conn.request("GET", "/ntp1/stakeholders/%7Btokenid%7D", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

GROUND TRUTH CODE:
import http.client

conn = http.client.HTTPSConnection("ntp1node.nebl.io")

conn.request("GET", "//ntp1/stakeholders/%7Btokenid%7D")

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


SAMPLE 2 | Exact Match: False | Syntax Valid: True
----------

In [ ]:
def generate_code_1(prompt, max_new_tokens=256):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            top_p=0.95,
            do_sample=False,
        )

    full = tokenizer.decode(output[0], skip_special_tokens=True)
    return full[len(prompt):].strip()


In [ ]:
print("\n" + "="*80)
print("SHOWING 5 DIVERSE EXAMPLES: PREDICTION vs GROUND TRUTH")
print("="*80 + "\n")

for idx, ex in enumerate(eval_samples[:5]):
    original_obj = ex["full_obj"]
    gold = ex["api_call"].strip()

    prompt = build_prompt(original_obj)
    pred = generate_code_1(prompt).strip()
    em = pred == gold
    syntax_ok = python_is_valid(pred)

    print(f"SAMPLE {idx+1} | Exact Match: {em} | Syntax Valid: {syntax_ok}")
    print("-" * 70)
    print("USER INSTRUCTION:")
    print(original_obj["input"].get("instruction", "").strip())
    print("\nENDPOINT:", original_obj["input"].get("path", "").strip())
    print("\nPREDICTED CODE:")
    print(pred)
    print("\nGROUND TRUTH CODE:")
    print(gold)
    print("\n" + "="*80 + "\n")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



SHOWING 5 DIVERSE EXAMPLES: PREDICTION vs GROUND TRUTH



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


SAMPLE 1 | Exact Match: False | Syntax Valid: True
----------------------------------------------------------------------
USER INSTRUCTION:
Please give me an example of how to use the endpoint getTokenHolders from Neblio REST API Suite to find out which addresses hold a specific token and how many tokens they have.

ENDPOINT: /ntp1/stakeholders/{tokenid}

PREDICTED CODE:
import http.client

conn = http.client.HTTPSConnection("ntp1node.neblio.org:10001")

conn.request("GET", "/ntp1/stakeholders/%7Btokenid%7D")

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

GROUND TRUTH CODE:
import http.client

conn = http.client.HTTPSConnection("ntp1node.nebl.io")

conn.request("GET", "//ntp1/stakeholders/%7Btokenid%7D")

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))




Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


SAMPLE 2 | Exact Match: False | Syntax Valid: True
----------------------------------------------------------------------
USER INSTRUCTION:
How do I utilize the AdPay API to associate a customer with a user account? Please provide an example with the necessary parameters for a successful API call to link a customer to the user using the put-customers-customerId endpoint.

ENDPOINT: /customers/{customerId}

PREDICTED CODE:
import http.client

conn = http.client.HTTPSConnection("virtserver.swaggerhub.com")

payload = "{\"id\":0,\"firstName\":\"string\",\"lastName\":\"string\",\"email\":\"string\",\"phone\":\"string\",\"address\":\"string\",\"city\":\"string\",\"state\":\"string\",\"zipCode\":\"string\",\"country\":\"string\",\"customerId\":0}"

headers = {
    'accept': "application/json",
    'content-type': "application/json"
    }

conn.request("PUT", "/JANIS_1/AdPay/1.0.0/customers/%7BcustomerId%7D", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("ut

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


SAMPLE 3 | Exact Match: False | Syntax Valid: True
----------------------------------------------------------------------
USER INSTRUCTION:
How can I effectively add a new inventory item using API mutualisme? Please provide an example with precise details on the necessary request including item name, description, and stock quantity.

ENDPOINT: /inventory

PREDICTED CODE:
import http.client

conn = http.client.HTTPSConnection("virtserver.swaggerhub.com")

payload = "{\"id\":\"d290f1ee-6c54-4b01-90e6-d701748f0851\",\"name\":\"Widget Adapter\",\"releaseDate\":\"2016-08-29T09:12:33.001Z\",\"manufacturer\":{\"name\":\"ACME Corporation\",\"homePage\":\"https://www.acme-corp.com\",\"phone\":\"408-867-5309\"}}"

headers = {
    'accept': "application/json",
    'content-type': "application/json"
    }

conn.request("POST", "/mutualisme/mutualisme/1.0.0/inventory", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

GROUND TRUTH CODE:
import http.client



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


SAMPLE 4 | Exact Match: False | Syntax Valid: True
----------------------------------------------------------------------
USER INSTRUCTION:
How do I send DICOM resources from my local storage to a remote Orthanc peer using the Orthanc API?

ENDPOINT: /peers/{id}/store

PREDICTED CODE:
import http.client

conn = http.client.HTTPSConnection("demo.orthanc-server.com")

payload = "{\"url\":\"string\"}"

headers = { 'content-type': "application/json" }

conn.request("POST", "//peers/%7Bid%7D/store", payload, headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

GROUND TRUTH CODE:
import http.client

conn = http.client.HTTPSConnection("demo.orthanc-server.com")

payload = "{\"Asynchronous\":true,\"Compress\":true,\"Permissive\":true,\"Priority\":0,\"Resources\":[\"string\"],\"Synchronous\":true,\"Transcode\":\"string\"}"

headers = { 'content-type': "application/json" }

conn.request("POST", "//peers/%7Bid%7D/store", payload, headers)

res = conn.getresponse()
da

In [ ]:
print("\n=== RUNNING EVALUATION ON 100 REAL HELD-OUT SAMPLES ===\n")

results = {
    "exact_match": [], "bleu": [], "sacrebleu": [], "rougeL": [], "perplexity": [], "syntax": []
}

for ex in eval_samples:
    original_obj = ex["full_obj"]
    gold = ex["api_call"]

    prompt = build_prompt(original_obj)
    pred = generate_code_1(prompt)


    results["exact_match"].append(exact_match(pred, gold))

    results["bleu"].append(bleu_metric.compute(predictions=[pred], references=[[gold]])["bleu"])

    results["sacrebleu"].append(sacrebleu_metric.compute(predictions=[pred], references=[gold])["score"])

    results["rougeL"].append(rouge_metric.compute(predictions=[pred], references=[gold])["rougeL"])

    results["perplexity"].append(compute_perplexity(pred))

    results["syntax"].append(int(python_is_valid(pred)))

print("=== FINAL HONEST METRICS (100 held-out Python samples) ===")
print(f"Exact Match     : {mean(results['exact_match']):.4f}")
print(f"BLEU            : {mean(results['bleu']):.4f}")
print(f"SacreBLEU       : {mean(results['sacrebleu']):.4f}")
print(f"ROUGE-L         : {mean(results['rougeL']):.4f}")
print(f"Perplexity      : {mean(results['perplexity']):.4f}")
print(f"Syntax Valid    : {mean(results['syntax']):.4f}")

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



=== RUNNING EVALUATION ON 100 REAL HELD-OUT SAMPLES ===



Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for

=== FINAL HONEST METRICS (100 held-out Python samples) ===
Exact Match     : 0.2300
BLEU            : 0.7642
SacreBLEU       : 76.4220
ROUGE-L         : 0.8139
Perplexity      : 2.1862
Syntax Valid    : 0.8800
